# Viterbi algorithm

In [ ]:
import numpy as np

In [ ]:
def viterbi(sentence, init_matrix, transition_matrix, emission_matrix, word_to_index, category_to_index, index_to_category):
    num_categories = len(init_matrix) # num categories N
    num_words = len(sentence) # num words in the sentence T

    # viterbi table
    viterbi_table = np.full((num_categories, num_words), -np.inf) # inf for logarithms
    backpointer_table = np.zeros((num_categories, num_words), dtype=int)


    # first step (init)
    for category in range(num_categories):
        word_index = word_to_index.get(sentence[0], word_to_index["UNK"]) # we use UNK tag if the word is not in the vocab
        viterbi_table[category, 0] = init_matrix[category] + emission_matrix[category, word_index] # sum because of logs
        

    # recursion step
    for t in range (1, num_words): # from 2 to T
        for category in range(num_categories): # from 1 to N
            max_prob = -np.inf
            max_prev_category = 0
            word_index = word_to_index.get(sentence[t], word_to_index["UNK"])

            for prev_category in range(num_categories):
                prev_viterbi = viterbi_table[prev_category, t-1]
                transition_score = transition_matrix[prev_category, category]
                '''
                if word_index is not None:
                    emission_score = emission_matrix[category, word_index]
                else:
                    emission_score = 0
                '''
                emission_score = emission_matrix[category, word_index]
                current_prob = prev_viterbi + transition_score + emission_score

                if current_prob > max_prob:
                    max_prob = current_prob
                    max_prev_category = prev_category

            # update tables viterbi and backpointer
            viterbi_table[category, t] = max_prob
            backpointer_table[category, t] = max_prev_category # a la que apunta 

    # final probability step
    max_final_category = np.argmax(viterbi_table[:, -1]) # all the rows of the last column
    max_prob = viterbi_table[max_final_category, -1] # maximum probability is the one in the last column of the max_final category row

    # construct optimal sequence
    optimal_seq = [max_final_category]
    for t in range(num_words-1, 0, -1): # from num_words-1 until 1, backward
        back_value = backpointer_table[optimal_seq[0], t]
        optimal_seq.insert(0, back_value)

    optimal_tags = [index_to_category[idx] for idx in optimal_seq]
    
    return optimal_tags, max_prob
